<a href="https://colab.research.google.com/github/VaibhavvvBarot/VaibhavvvBarot/blob/main/JsonCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *JsonOptimizer*

*   Assign
*   Conditional



In [26]:
import json
from google.colab import output
from IPython.display import display, HTML

In [38]:
# Define the valid keys for each type
LOGIC_CONTROL_KEYS = [
    "ValueType",
    "DocSourceName",
    "Value",
    "DataType",
    "ValueSubType",
    "DataSubType",
    "LookupType",
    "LookupName"
]

LOGIC_EXPRESSION_ADDITIONAL_KEYS = [
    "ConditionalOperator",
    "ID",
    "ExpressionType",
    "CompareToValue",
    "CompareToValueType",
    "CompareToDocSourceName",
    "CompareToDataType",
    "CompareToDataSubType",
    "CompareToLookupType",
    "LogicalOperator"
]

# Action-specific keys
ASSIGN_VALUE_OUTPUT_KEYS = ["AssignAction"]
HIDE_SHOW_ENABLE_KEYS = ["Index"]

# Keys to always include for any logic action
COMMON_LOGIC_ACTION_KEYS = ["logicAction"]

def filter_logic_control(item):
    """Filter item to only include valid LogicControl keys that are present"""
    filtered_item = {}
    for key in LOGIC_CONTROL_KEYS:
        if key in item:
            filtered_item[key] = item[key]
    return filtered_item

def filter_logic_expression(item):
    """Filter item to include both LogicControl and LogicExpression keys that are present"""
    filtered_item = filter_logic_control(item)
    for key in LOGIC_EXPRESSION_ADDITIONAL_KEYS:
        if key in item:
            filtered_item[key] = item[key]
    return filtered_item

def optimize_logic_action(input_json):
    """Recursively optimize a logic action of any type"""
    if not isinstance(input_json, dict):
        return input_json

    # Keep track of the action type
    logic_action = input_json.get("logicAction", "")

    # Initialize the result with the required keys
    result = {"logicAction": logic_action}

    # Process based on the specific logicAction type
    if logic_action == "Assign Value":
        # Process logicInstanceInputs
        if "logicInstanceInputs" in input_json:
            result["logicInstanceInputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceInputs"]
            ]

        # Process logicInstanceOutputs
        if "logicInstanceOutputs" in input_json:
            outputs = []
            for item in input_json["logicInstanceOutputs"]:
                filtered_item = filter_logic_control(item)
                # Add action-specific keys
                for key in ASSIGN_VALUE_OUTPUT_KEYS:
                    if key in item:
                        filtered_item[key] = item[key]
                outputs.append(filtered_item)
            result["logicInstanceOutputs"] = outputs

    elif logic_action == "Conditional":
        # Process logicExpression
        if "logicExpression" in input_json:
            result["logicExpression"] = [
                filter_logic_expression(item) for item in input_json["logicExpression"]
            ]

        # Process nested logicInstanceTrue - maintain the structure with "logicInstance"
        if "logicInstanceTrue" in input_json:
            true_instances = []
            for item in input_json["logicInstanceTrue"]:
                if "logicInstance" in item:
                    # Recursively optimize the nested logicInstance
                    optimized_instance = optimize_logic_action(item["logicInstance"])
                    true_instances.append({"logicInstance": optimized_instance})
                else:
                    # If it doesn't have a logicInstance, keep it as is
                    true_instances.append(item)
            result["logicInstanceTrue"] = true_instances

        # Process nested logicInstanceFalse - maintain the structure with "logicInstance"
        if "logicInstanceFalse" in input_json:
            false_instances = []
            for item in input_json["logicInstanceFalse"]:
                if "logicInstance" in item:
                    # Recursively optimize the nested logicInstance
                    optimized_instance = optimize_logic_action(item["logicInstance"])
                    false_instances.append({"logicInstance": optimized_instance})
                else:
                    # If it doesn't have a logicInstance, keep it as is
                    false_instances.append(item)
            result["logicInstanceFalse"] = false_instances

    # Handle Hide Fields, Show Fields, Enable Fields and similar actions
    elif logic_action in ["Hide Fields", "Show Fields", "Enable Fields"]:
        if "logicInstanceInputs" in input_json:
            inputs = []
            for item in input_json["logicInstanceInputs"]:
                filtered_item = filter_logic_control(item)
                # Add action-specific keys
                for key in HIDE_SHOW_ENABLE_KEYS:
                    if key in item:
                        filtered_item[key] = item[key]
                inputs.append(filtered_item)
            result["logicInstanceInputs"] = inputs

        # These actions typically don't have outputs, but handle it just in case
        if "logicInstanceOutputs" in input_json and input_json["logicInstanceOutputs"]:
            result["logicInstanceOutputs"] = input_json["logicInstanceOutputs"]

    # For any other logic action type, just keep the basic structure
    else:
        # Handle inputs if present
        if "logicInstanceInputs" in input_json:
            result["logicInstanceInputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceInputs"]
            ]

        # Handle outputs if present
        if "logicInstanceOutputs" in input_json:
            result["logicInstanceOutputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceOutputs"]
            ]

        # Handle nested true/false branches if present
        for branch in ["logicInstanceTrue", "logicInstanceFalse"]:
            if branch in input_json:
                branch_instances = []
                for item in input_json[branch]:
                    if "logicInstance" in item:
                        optimized_instance = optimize_logic_action(item["logicInstance"])
                        branch_instances.append({"logicInstance": optimized_instance})
                    else:
                        branch_instances.append(item)
                result[branch] = branch_instances

    return result

# Get JSON input from user
json_text = input("Paste your JSON here: ")

try:
    input_json = json.loads(json_text)
    optimized_json = optimize_logic_action(input_json)

    # Format the optimized JSON
    formatted_json = json.dumps(optimized_json, indent=2)

    # Clear previous output
    output.clear()

    # Create a text area with the optimized JSON for easy copying
    display(HTML(f"""
    <h3>Optimized JSON (Click inside the box, press Ctrl+A to select all, then Ctrl+C to copy):</h3>
    <textarea id="json-output" style="width:100%; height:500px; font-family:monospace; white-space:pre;">{formatted_json}</textarea>
    <script>
        // Auto-select the text for easy copying
        var textarea = document.getElementById('json-output');
        textarea.focus();
        textarea.select();
    </script>
    """))

    print("\nThe optimized JSON is ready to copy from the text area above.")

except Exception as e:
    print(f"Error: {str(e)}")


The optimized JSON is ready to copy from the text area above.
